In [1]:
#Import Dependentcies

from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [21]:

# print(soup.prettify())

In [7]:
#Scrape News title

news_title = soup.find('div', class_='content_title').find('a').text.strip()


news_title

"NASA's InSight Places First Instrument on Mars"

In [11]:
#Scrape body

news_p = soup.find_all(class_="rollover_description_inner")[0].text.strip()


news_p

'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.'

In [2]:
#Splinter setup for Windows
# Chromedriver is set as a Windows path variable

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Featured image url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url)




In [13]:
html = browser.html

soup = BeautifulSoup(html, 'html.parser')

url_1 = 'https://www.jpl.nasa.gov'

url_2 = soup.find_all('img', class_='thumb')[0]['src']

featured_image_url = url_1 + url_2

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22904-640x350.jpg'

In [14]:
#Mars weather

# Visit the mars weather twitter account
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)


soup = BeautifulSoup(response.text, 'html.parser')

# Scrape the twitter page for the most recent Mars weather
mars_weather = soup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')[0].text

mars_weather

'Sol 2270 (2018-12-25), high -5C/23F, low -75C/-102F, pressure at 8.35 hPa, daylight 06:41-18:53'

In [23]:
# Mars Facts

# Visit Mars facts url 
facts_url = 'https://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

# Assign the column headers
mars_df.columns = ['Description','Value']

#  Set the index to the `Description` column
mars_df.set_index('Description', inplace=True)

mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [38]:
# Mars Hemisphere images
# NOTE: Due to the astrogeology website being down, a cached version from web.archive.org had to be used.

hemispheres_url = 'http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)

# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

items



In [35]:
# Append the URL to the cached version of the image site
new_url = 'http://web.archive.org' + items

browser.visit(new_url)

html_hemispheres = browser.html

soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retrieve the url for the full jpeg image
hemi_img = soup.find('div', class_='wide-image-wrapper').find('li').find('a')['href']

hemi_img



'http://web.archive.org/web/20181114182238/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [37]:
# Loop for gathering of hemisphere img urls and titles

# Define list to store the image urls
hemisphere_image_urls = []

for x in items: 
    # Store title
    title = x.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = x.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    new_url = 'http://web.archive.org' + partial_img_url
    
    browser.visit(new_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    hemi_img = soup.find('div', class_='wide-image-wrapper').find('li').find('a')['href']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : hemi_img})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182238/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182242/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182245/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://web.archive.org/web/20181114182248/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]